<a href="https://colab.research.google.com/github/sdam-au/PHI_ETL/blob/master/scripts/1_3_EXTRACTING_DATES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math
import pandas as pd
import re

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET

import zipfile
import io

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
!pip install sddk
import sddk

In [3]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [0]:
### authorize google sheets
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
# establish connection with particular sheet by its url:
PHI_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1zfTw0Hf304maBmrYvaMxRLnv1zfAVFixrtGTTsLCcT4/edit?usp=sharing")

# Read data

In [0]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_enriched_raw.json", "df", conf)

In [6]:
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID
0,[/text/237443?location=1365&patt=&bookid=411&b...,[CSCA],"[5 (1972) 169,3]",[Regions\nAttica (IG I-III)\nAttica],"[CSCA\n5 (1972) 169,3]",[Att. — Athens: Akropolis — stoich. 28 — 303/2...,{},[4],[1\n\n\n],[ἐπὶ Λε[ωστράτου ἄρχοντος ἐπὶ τῆς Κ]-\nεκρο[πί...,[CSCA.csv],[237443]
1,[/text/237444?location=1365&patt=&bookid=411&b...,[CSCA],"[5 (1972) 169,4]",[Regions\nAttica (IG I-III)\nAttica],"[CSCA\n5 (1972) 169,4]",[Att. — Athens: EM — stoich. 35 — 306-302 BC],{},[15],[1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15],[[․]ομε̣[․․․․․․․․․․․․․28․․․․․․․․․․․․․ τῆς]\nπρ...,[CSCA.csv],[237444]
2,[/text/237445?location=1365&patt=&bookid=411&b...,[CSCA],"[5 (1972) 173,5]",[Regions\nAttica (IG I-III)\nAttica],"[CSCA\n5 (1972) 173,5]",[Att. — Athens — non-stoich.],{},[16],[1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n],[— — — — —υτω․․․․11․․․․․\n— — — — —ωντωι̣․․․․9...,[CSCA.csv],[237445]
3,[/text/237446?location=1365&patt=&bookid=411&b...,[CSCA],"[9 (1976) 9,4]",[Regions\nAttica (IG I-III)\nAttica],"[CSCA\n9 (1976) 9,4]",[Att. — Athens: Eth.Mus. — 4th c. BC — SEG 26....,{},[2],[A.1\nB.1],[ψῆφος ⋮ δημοσία.\nΔ ․],[CSCA.csv],[237446]
4,[/text/237447?location=1365&patt=&bookid=411&b...,[CSCA],"[9 (1976) 11,16]",[Regions\nAttica (IG I-III)\nAttica],"[CSCA\n9 (1976) 11,16]",[Att. — Athens: Eth.Mus. — 4th c. BC — SEG 26....,{},[2],[A.1\nB.1],[ψῆφος ⋮ δημοσία.\nΓ Ε],[CSCA.csv],[237447]


In [0]:
def lists_to_values(list_or_value):
  if isinstance(list_or_value, list):
    value = list_or_value[0]
  else: 
    value = list_or_value
  return value
for column in PHI.columns:
  PHI[column] = PHI.apply(lambda row: lists_to_values(row[column]), axis=1)

In [8]:
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID
0,/text/237443?location=1365&patt=&bookid=411&bo...,CSCA,"5 (1972) 169,3",Regions\nAttica (IG I-III)\nAttica,"CSCA\n5 (1972) 169,3",Att. — Athens: Akropolis — stoich. 28 — 303/2 BC,{},4,1\n\n\n,ἐπὶ Λε[ωστράτου ἄρχοντος ἐπὶ τῆς Κ]-\nεκρο[πίδ...,CSCA.csv,237443
1,/text/237444?location=1365&patt=&bookid=411&bo...,CSCA,"5 (1972) 169,4",Regions\nAttica (IG I-III)\nAttica,"CSCA\n5 (1972) 169,4",Att. — Athens: EM — stoich. 35 — 306-302 BC,{},15,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15,[․]ομε̣[․․․․․․․․․․․․․28․․․․․․․․․․․․․ τῆς]\nπρυ...,CSCA.csv,237444
2,/text/237445?location=1365&patt=&bookid=411&bo...,CSCA,"5 (1972) 173,5",Regions\nAttica (IG I-III)\nAttica,"CSCA\n5 (1972) 173,5",Att. — Athens — non-stoich.,{},16,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n,— — — — —υτω․․․․11․․․․․\n— — — — —ωντωι̣․․․․9․...,CSCA.csv,237445
3,/text/237446?location=1365&patt=&bookid=411&bo...,CSCA,"9 (1976) 9,4",Regions\nAttica (IG I-III)\nAttica,"CSCA\n9 (1976) 9,4",Att. — Athens: Eth.Mus. — 4th c. BC — SEG 26.1...,{},2,A.1\nB.1,ψῆφος ⋮ δημοσία.\nΔ ․,CSCA.csv,237446
4,/text/237447?location=1365&patt=&bookid=411&bo...,CSCA,"9 (1976) 11,16",Regions\nAttica (IG I-III)\nAttica,"CSCA\n9 (1976) 11,16",Att. — Athens: Eth.Mus. — 4th c. BC — SEG 26.1...,{},2,A.1\nB.1,ψῆφος ⋮ δημοσία.\nΓ Ε,CSCA.csv,237447


# Old version of date extraction

In [0]:
def date_extractor(date_value):
    ### simple cleaning?
    datation = re.sub("\/\d+", "", date_value)
    for element in ["?"]:
        datation = datation.replace(element, "")
    for element in ["AD", "Rom. Imp.", "Byzant", "Roman period", "Christian"]:
        if element in datation:
            for element_cent in [("1st", 1, 100),
                            ("2nd", 101, 200),
                            ("3rd", 201, 300),
                            ("4th", 301, 400),
                           ]:
                if element_cent[0] in datation:
                    dating = [element_cent[1], element_cent[2], None, round(np.average([element_cent[1], element_cent[2]])), "cent", "AD"]
                    break
                else:
                    try:
                        post = re.search('(after\s|post\s)(\d+)', datation, flags=re.IGNORECASE).groups()
                        post = int(post[1])
                        dating = [post, None, None, None, "post", "AD"]
                    except:
                        try:
                            date_both = re.search('(\d+)(\-)(\d+)', datation, flags=re.IGNORECASE).groups()
                            post = int(date_both[0])
                            ante = int(date_both[2])
                            dating = [post, ante, None, int(np.average([post, ante])), "range", "AD"]
                        except:
                            try:
                                date_exact = re.search('\d+', datation, flags=re.IGNORECASE)
                                date_exact = int(date_exact.group(0))
                                dating = [None, None, date_exact, date_exact, "exact", "AD"]
                            except:
                                dating = [None, None, None, None, "unprecise", "AD"]
                                
            break
        else:
            for element_cent in [(" V ", -500, -401), 
                            (" V/IV ", -500, -301),
                            (" IV ", -400, -301),
                            (" III ", -300, -201),
                            (" II ", -200, -101),
                            (" I ", -100, -1),
                            ("5th", -500,  -401),
                            ("4th", -400, -301),
                            ("3rd", -300, -201),
                            ("2nd", -200, -101),
                            ("1st", -100, -1)]:
                if element_cent[0] in datation:
                    ### if there as an element indicating century
                    dating = [element_cent[1], element_cent[2], None, int(np.average([element_cent[1], element_cent[2]])), "cent", "BC"]
                    break
                else:
                    try:
                        ante = re.search('(ante\s)(\d+)', datation, flags=re.IGNORECASE).groups()
                        ante = int(ante[1]) * -1
                        dating =  [None, ante, None, None, "ante", "BC"]
                    except:
                        try:
                            post = re.search('(post\s|after\s)(\d+)', datation, flags=re.IGNORECASE).groups()
                            post = int(post[1]) * -1
                            dating = [post, None, None, None, "post", "BC"]
                        except:
                            try:
                                date_both = re.search('(\d+)(\-)(\d+)', datation, flags=re.IGNORECASE).groups()
                                post = int(date_both[0]) * -1 
                                ante = int(date_both[2]) * -1
                                dating = [post, ante, None, int(np.average([post, ante])), "range", "BC"]
                            except:
                                try:
                                    date_exact = re.search('\d+', datation, flags=re.IGNORECASE)
                                    date_exact = int(date_exact.group(0)) * -1
                                    dating = [None, None, date_exact, date_exact, "exact", "BC"]
                                except:
                                        dating = [None, None, None, None, "string", "BC"]
    for reign in [("reign of Hadrian", 117, 138),
                  ("reign of Justinian", 527, 565),
                  ("reign of Ant. Pius", 138, 161),
                  ("reign of Augustus", -27, 19),
                  ("reign of Tiberius", 14, 37)]:
        if reign[0] in datation:
            avr = int(np.average([reign[1], reign[2]]))
            if avr < 0:
                period = "BC"
            else:
                period = "AD"
            dating = [reign[1], reign[2], None, avr, "range", period]
            break
    return [date_value] + dating 

In [0]:
#examples
datations = ["reign of Claudius", "Roman imperial period", "Hellenistic"]
for datation in datations:
  print(extract_period(datation))

['reign of Claudius', 41, 54.0, None, None, 47.5, 'range', 'AD']
['Roman imperial period', -31, 410.0, None, None, 189.5, 'range', 'AD']
['Hellenistic', -323, -31.0, None, None, -177.0, 'range', 'BC']


# Raw date column

In [0]:
def get_date_from_tildeinfo(tildeinfo):
  try:
    tildeinfo_list = tildeinfo.split("—")
    datation = tildeinfo_list[-1]
    for el in tildeinfo_list:
      if any(time_indicator in el for time_indicator in ["BC", "AD", "period", "reign"]):
        datation = el
        break
  except: 
    datation = ""
  return datation 

In [0]:
PHI["raw_date"] = PHI.apply(lambda row: get_date_from_tildeinfo(row["tildeinfo"]), axis=1)

# Testing extracting info from tildeinfo

In [11]:
# generate sample for testing purposes:
PHI_by_500 = PHI[PHI["PHI_ID"].isin(range(1, 300000, 500))]
PHI_by_500.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID,raw_date
625,/text/262001?location=1040&patt=&bookid=517&of...,Panamara,316,Regions\nAsia Minor\nCaria,Panamara\n316,IStr 427,{},2,1\n,"Διῒ Πανημέρῳ, κόμαι Εὐτύχεως· ἐπὶ ἱερέω[ς] Κλα...",Panamara.csv,262001,IStr 427
912,/text/266001?location=1673&patt=&bookid=587&of...,St.Pont. III,110a,Regions\nAsia Minor\nPontus and Paphlagonia,St.Pont. III\n110a,Pont. — Amasia — Rom. Imp. period,{},6,1\n\n\n\n5\n,Αὐρ(ηλίῳ) Φιλο-\nμούσῳ ❦\nἀρχιάτρῳ\nῬω[μᾶνα(?)...,St.Pont.-III.csv,266001,Rom. Imp. period
1260,/text/231001?location=1365&patt=&bookid=394&of...,Agora XV,130,Regions\nAttica (IG I-III)\nAttica,Agora XV\n130,Att. — Athens: Agora — 220/19,{},149,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20...,ἐπὶ Μενεκράτου ἄρχοντος ἐπὶ τῆς Οἰνεῖδος ἕκτη-...,Agora-XV.csv,231001,220/19
1944,/text/79501?location=126&patt=&bookid=23&offse...,"IG XII,8",311,"Regions\nAegean Islands, incl. Crete (IG XI-[X...","IG XII,8\n311",Thasos,{},7,1\n\n\n\n5\n\n,Ἐπικράτης Κτησιφῶντος\nΠυθίων Περιθύμου\nΠίν...,IG-XII-8.csv,79501,Thasos
2502,/text/218501?location=1497&patt=&bookid=367&of...,Koptos à Kosseir,163,"Regions\nEgypt, Nubia and Cyrenaïca\nEgypt and...",Koptos à Kosseir\n163,Eg. — el-Boueib,{},2,1\n,Κάλχων̣\nΠανὶ Χρ(υσόδωτηι).,Koptos---Kosseir.csv,218501,el-Boueib


# Parse ante quem and post quem

In [12]:
### simple demonstration of the logic
datation = "not before 304 AD"
match = re.search("(not\s(before|bef\.)\s|non\sante\s)(\d+)",  datation, flags=re.IGNORECASE)
if match:
  dating_update = {"start" : int(match.groups()[2]), "type" : "post"}
dating_update

{'start': 304, 'type': 'post'}

In [0]:
def extract_ante_and_post(datation, dating):
  if "unknown" in dating["type"]: 
    # if "NOT BEFORE"
    match = re.search("(not\s(before|bef\.)\s|non\sante\s)(\-?\d+)(\s|$)",  datation, flags=re.IGNORECASE)
    if match:
      if "AD" not in datation:
        start = (int(match.groups()[2]) * -1) - 1
        dating_update = {"start" : start, "type" : "post"}
      else:
        dating_update = {"start" : int(match.groups()[2]) - 1, "type" : "post"}
    # if "BEFORE"
    else:
      match = re.search('(before\s|ante\s)(\-?\d+)(\s|$)', datation, flags=re.IGNORECASE)
      if match:
        dating_update = {"stop" : int(match.groups()[1]), "type" : "ante"}
        if "AD" not in datation:
          dating_update["stop"] = dating_update["stop"] * -1
      # if "NOT AFTER"
      else:
        match = re.search("(not\safter\s|non\spost\s)(\-?\d+)(\s|$)",  datation, flags=re.IGNORECASE)
        if match:
          if "AD" not in datation:
            dating_update = {"stop" : (int(match.groups()[1]) * -1) + 1, "type" : "ante"}
          else:
            dating_update = {"stop" : int(match.groups()[1]) + 1, "type" : "ante"}
        # if "AFTER"
        else:
            match = re.search('(after\s|post\s)(\-?\d+)(\s|$)', datation, flags=re.IGNORECASE)
            if match:
              dating_update = {"start" : int(match.groups()[1]), "type" : "post"}
              if "AD" not in datation:
                dating_update["start"] = dating_update["start"] * -1
            else:
              dating_update = dating
  elif "exact+or" in dating["type"]: 
    # if "NOT BEFORE"
    match = re.search("(not\s(before|bef\.)\s|non\sante\s)",  datation, flags=re.IGNORECASE)
    if match:
      dating_update = {"start" : dating["exact"], "or": {"start" : dating["or"]["exact"], "exact" : None}, "exact" : None, "type" : "post+or"}
    # if "BEFORE"
    else:
      match = re.search('(before\s|ante\s)', datation, flags=re.IGNORECASE)
      if match:
        dating_update = {"stop" : dating["exact"], "or": {"stop" : dating["or"]["exact"], "exact" : None}, "exact" : None, "type" : "ante+or"}
      # if "NOT AFTER"
      else:
        match = re.search("(not\safter\s|non\spost\s)",  datation, flags=re.IGNORECASE)
        if match:
              dating_update = {"stop" : dating["exact"], "or": {"stop" : dating["or"]["exact"], "exact" : None}, "exact" : None, "type" : "ante+or"}
        # if "AFTER"
        else:
            match = re.search('(after\s|post\s)', datation, flags=re.IGNORECASE)
            if match:
              dating_update = {"start" : dating["exact"], "or": {"start" : dating["or"]["exact"], "exact" : None}, "exact" : None, "type" : "post+or"}
            else:
              dating_update = dating
  elif "range" in dating["type"]:
    # if "NOT BEFORE"
    match = re.search("(not\s(before|bef\.)\s|non\sante\s)",  datation, flags=re.IGNORECASE)
    if match:
      dating_update = {"start" : dating["start"] - 1, "stop":None, "type" : dating["type"]+"+post"}
    # if "BEFORE"
    else:
      match = re.search('(before\s|ante\s)', datation, flags=re.IGNORECASE)
      if match:
        dating_update = {"stop" : dating["start"], "start":None, "type" : dating["type"]+"+ante"}  
      # if "NOT AFTER"
      else:
        match = re.search("(not\safter\s|non\spost\s)",  datation, flags=re.IGNORECASE)
        if match:
          dating_update = {"stop" : dating["stop"] + 1, "start":None,"type" : dating["type"]+"+ante"}
        # if "AFTER"
        else:
          match = re.search('(after\s|post\s)', datation, flags=re.IGNORECASE)
          if match:
            dating_update = {"start" : dating["stop"], "stop":None,"type" : dating["type"]+"+post"}
          else:
            dating_update = dating
  else: 
    #datation = re.sub("(not\s(after|bef\.)\s|non\spost)", "ante\s", datation)
    dating_update = dating
  if "shortly" in datation:
    dating_update["type"] = dating_update["type"] + "+shortly"
  return dating_update

In [14]:
# example with "unknown"
dating = {"type" : "unknown", "era": None}
for datation in ["non post 230 AD", "shortly after 320 BC", "not bef. 114 BC","not after 317 AD", "before 200", "Ante 114 BC", "post 2nd century BC"]:
  print(datation, extract_ante_and_post(datation, dating))

non post 230 AD {'stop': 231, 'type': 'ante'}
shortly after 320 BC {'start': -320, 'type': 'post+shortly'}
not bef. 114 BC {'start': -115, 'type': 'post'}
not after 317 AD {'stop': 318, 'type': 'ante'}
before 200 {'stop': -200, 'type': 'ante'}
Ante 114 BC {'stop': -114, 'type': 'ante'}
post 2nd century BC {'type': 'unknown', 'era': None}


In [15]:
datation, dating = "not after reign of Trajan", {"start": 98, "stop" : 117, "type" : "range+period", "era": None}
extract_ante_and_post(datation, dating)

{'start': None, 'stop': 118, 'type': 'range+period+ante'}

# Parse periods

In [16]:
# read periods from our external code
periods = get_as_dataframe(PHI_overview.worksheet("periods"))
periods

,period,start,stop,type,era,source,notes,link
0,Roman imp,-31,410,range+period,AD,PeriodO,NaN,http://n2t.net/ark:/99152/p08m57hqcc5
1,Rom. Imp,-31,410,range+period,BC/AD,PeriodO,NaN,http://n2t.net/ark:/99152/p08m57hqcc5
2,aet. imp.,-31,410,range+period,BC/AD,PeriodO,NaN,http://n2t.net/ark:/99152/p08m57hqcc5
3,aet. Rom.,-146,324,range+period,BC/AD,NaN,NaN,NaN
4,Roman period,-146,324,range+period,BC/AD,NaN,NaN,NaN
5,reign of Hadrian,117,138,range+period,AD,PeriodO,NaN,http://n2t.net/ark:/99152/p0jrrjbntfj
6,reign of Justinian,527,565,range+period,AD,PeriodO,NaN,http://n2t.net/ark:/99152/p06c6g3r7ht
7,reign of Ant. Pius,138,161,range+period,AD,PeriodO,NaN,http://n2t.net/ark:/99152/p06c6g3drk4
8,reign of Augustus,-27,14,range+period,BC/AD,PeriodO,NaN,http://n2t.net/ark:/99152/p06c6g3xnmx
9,reign of Tiberius,14,37,range+period,AD,PeriodO,NaN,http://n2t.net/ark:/99152/p0jrrjbts8w


In [17]:
periods_dict = periods.set_index("period").T.to_dict()
periods_dict["reign of Claudius"]

{'era': 'AD',
 'link': 'http://n2t.net/ark:/99152/p0jrrjb8spw',
 'notes': nan,
 'source': 'PeriodO',
 'start': 41,
 'stop': 54,
 'type': 'range+period'}

In [0]:
def extract_period(datation, dating=None):
  if (dating==None) or (dating["type"] == "unknown"):
    for key in periods_dict.keys():
      if key.lower() in datation.lower(): # use lower cases to match everything
        period = periods_dict[key]   
        dating_update = period
        break
      else:
        dating_update = {"type" : "unknown"}
    return dating_update
  else:
    return dating


In [19]:
# example:
dating = {"type": "unknown"}
for datation in ["Byzantine", "Roman Imperial", "reign of Augustus", "140", "late Antonine period"]:
  print({datation : extract_period(datation, dating)})

{'Byzantine': {'type': 'unknown'}}
{'Roman Imperial': {'start': -31, 'stop': 410, 'type': 'range+period', 'era': 'AD', 'source': 'PeriodO', 'notes': nan, 'link': 'http://n2t.net/ark:/99152/p08m57hqcc5'}}
{'reign of Augustus': {'start': -27, 'stop': 14, 'type': 'range+period', 'era': 'BC/AD', 'source': 'PeriodO', 'notes': nan, 'link': 'http://n2t.net/ark:/99152/p06c6g3xnmx'}}
{'140': {'type': 'unknown'}}
{'late Antonine period': {'start': 96, 'stop': 192, 'type': 'range+period', 'era': 'AD', 'source': 'PeriodO', 'notes': nan, 'link': 'http://n2t.net/ark:/99152/p06c6g34zjk'}}


# EDH periods exploratations
(skip, not needed)

In [0]:
# read periods from our external code
EDH_periods = get_as_dataframe(PHI_overview.worksheet("EDH_periods"))
EDH_periods.head(5)

,period,label,spatial_coverage,gazetteer_links,start,stop,authority,source,publication_year,derived_periods
0,http://n2t.net/ark:/99152/p0jrrjb7g67,Augustan Age,Roman Empire,NaN,-26,14,http://n2t.net/ark:/99152/p0jrrjb,Epigraphische Datenbank Heidelberg | Heidelber...,2017,NaN
1,http://n2t.net/ark:/99152/p0jrrjbw83z,Julio-Claudian Dynasty,Roman Empire,NaN,-26,68,http://n2t.net/ark:/99152/p0jrrjb,Epigraphische Datenbank Heidelberg | Heidelber...,2017,NaN
2,http://n2t.net/ark:/99152/p0jrrjbvfhm,Roman Empire,Roman Empire,NaN,-26,476,http://n2t.net/ark:/99152/p0jrrjb,Epigraphische Datenbank Heidelberg | Heidelber...,2017,NaN
3,http://n2t.net/ark:/99152/p0jrrjbts8w,Tiberian,Roman Empire,NaN,14,37,http://n2t.net/ark:/99152/p0jrrjb,Epigraphische Datenbank Heidelberg | Heidelber...,2017,NaN
4,http://n2t.net/ark:/99152/p0jrrjb8spw,Claudian,Roman Empire,NaN,41,54,http://n2t.net/ark:/99152/p0jrrjb,Epigraphische Datenbank Heidelberg | Heidelber...,2017,NaN


In [0]:
def count_instances(label):
  return len(PHI[PHI["raw_date"].str.contains(label)])

EDH_periods["label_in_PHI"] = EDH_periods.apply(lambda row: count_instances(row["label"]), axis=1)

In [0]:
PHI[PHI["raw_date"].str.contains("Trajanic")]

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID,raw_date
150830,/text/270547?location=254&patt=&bookid=618&off...,MAMA IX,223,Regions\nAsia Minor\nPhrygia,MAMA IX\n223,"Phryg., N. — Aizanoi (Çavdarhisar) — prob. Tra...",{},2,2\n1,Μενέλ̣α̣ος Μ̣ε̣ν̣ελάῳ [τρό]φ̣ιμος θρέψαντι\nμν...,MAMA-IX.csv,270547,prob. Trajanic


In [0]:
set_with_dataframe(PHI_overview.add_worksheet("EDH_periods_in_PHI", 1, 1), EDH_periods)

# Parse "/" for individual dates

In [0]:
def match_or(datation, dating):
  if dating["type"] == "unknown":
    match = re.search(r'(\d+)(\/)(\d+)', str(datation), flags=re.IGNORECASE)
    if match != None:
        date1 = match.groups()[0]
        date2 = match.groups()[2]
        len_diff = len(date1) - len(date2)
        # complete the numbers 
        if len_diff > 0:
          date2 = date1[:len_diff] + date2
        # transform it into integer
        date1 = int(date1)
        date2 = int(date2)
        if "AD" not in datation:
          date1 = date1 * -1
          date2 = date2 * -1
          dating["era"] = "BC"
        else:
          dating["era"] = "AD"
        if date1 > date2:
          date1, date2 = date2, date1
        if abs(date1 - date2) < 2: # if these are neighboring numbers
          dating.update({"start" : date1, "stop": date2, "type" : "range"})
        else:
          dating.update({"exact" : date1, "or": {"exact" : date2, "type" : "exact"}, "type" : "exact+or"})
        dating = extract_ante_and_post(datation, dating)
        return dating
        #if dating_update["type"] == "post":
        #   return {"start" : date1, "or": {"start" : date2, "type" : "post"}, "type" : "post+or"}
        #elif extract_ante_and_post(datation, dating)["type"] == "ante":
        #  return {"stop" : date1, "or": {"stop" : date2, "type" : "ante"}, "type" : "ante+or"}
        #else:
        #  return {"exact" : date1, "or": {"exact" : date2, "type" : "exact"}, "type" : "exact+or"}
    else:
      return {"type" : "unknown"}
  else:
    return {"type" : "unknown"}

In [21]:
dating = {"type": "unknown"}
match_or(" 12/1 BC", dating)

{'era': 'BC', 'start': -12, 'stop': -11, 'type': 'range'}

In [22]:
ors = PHI_by_500[PHI_by_500["raw_date"].str.contains(r'(\d+)(\/)(\d+)')]["raw_date"].tolist()
print(ors)

[' 220/19', ' 12/1 BC', ' 116/5 BC', ' 229/30 or 230/1 AD ', ' ante 336/5', ' 204/3?', ' 73/4 AD', ' ca. 163/2 BC', ' 128/127 BC ', ' shortly after 208/7 BC (or 207/6) ', ' 154/5 AD', ' 66/7 AD']


/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [23]:
datation = "12/1 BC"
match = re.search(r'(\d+)(\/)(\d+)', str(datation), flags=re.IGNORECASE)
if match:
  date1 = match.groups()[0]
  date2 = match.groups()[2]
  len_diff = len(date1) - len(date2)
  # complete the numbers 
  if len_diff > 0:
    date2 = date1[:len_diff] + date2
  # transform it into integer
  date1 = int(date1)
  date2 = int(date2)
print(date1, date2)

12 11


In [24]:
match_or(" 12/1 BC", {"type":"unknown"})

{'era': 'BC', 'start': -12, 'stop': -11, 'type': 'range'}

In [25]:
# example/testing
dating = {"type": "unknown"}
for our_or in ors:
  print({our_or: match_or(our_or, dating)})
  #print({our_or: extract_ante_and_post(our_or)})

{' 220/19': {'type': 'range', 'era': 'BC', 'start': -220, 'stop': -219}}
{' 12/1 BC': {'type': 'unknown'}}
{' 116/5 BC': {'type': 'unknown'}}
{' 229/30 or 230/1 AD ': {'type': 'unknown'}}
{' ante 336/5': {'type': 'unknown'}}
{' 204/3?': {'type': 'unknown'}}
{' 73/4 AD': {'type': 'unknown'}}
{' ca. 163/2 BC': {'type': 'unknown'}}
{' 128/127 BC ': {'type': 'unknown'}}
{' shortly after 208/7 BC (or 207/6) ': {'type': 'unknown'}}
{' 154/5 AD': {'type': 'unknown'}}
{' 66/7 AD': {'type': 'unknown'}}


In [26]:
ors = PHI_by_500[PHI_by_500["raw_date"].str.contains("/")]["raw_date"].tolist()
print(ors)

[' 220/19', ' 2nd/3rd c. AD', ' 12/1 BC', ' 2nd/3rd c. AD ', ' 116/5 BC', ' 3rd/2nd c. BC?', ' 6th/5th c. BC', ' 6th/5th c. BC', ' 229/30 or 230/1 AD ', ' 2nd/3rd c. AD ', ' 3rd/4th c. AD ', ' ante 336/5', ' 204/3?', ' [s. II/III p.]', ' 73/4 AD', ' 1st/2nd c. AD', ' fin. s. V/VI p.', ' 5th/6th c. AD', ' 2nd/3rd c. AD', ' ca. 163/2 BC', ' 3rd/2nd c. BC', ' 1st/2nd c. AD', ' 2nd/3rd c. AD ', ' 128/127 BC ', ' 2nd/3rd c. AD ', ' 2nd/1st c. BC', ' 1st/2nd c. AD', ' 3rd/4th c. AD? ', ' 2nd/3rd c. AD', ' 1st/2nd c. AD', ' Laurent, Corpus V 357bis/1702', ' shortly after 208/7 BC (or 207/6) ', ' 154/5 AD', ' 66/7 AD', ' 1st c. BC/1st c. AD ', ' 2nd/3rd c. AD']


In [27]:
PHI_by_500[PHI_by_500["raw_date"].str.contains("/")].head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID,raw_date
1260,/text/231001?location=1365&patt=&bookid=394&of...,Agora XV,130,Regions\nAttica (IG I-III)\nAttica,Agora XV\n130,Att. — Athens: Agora — 220/19,{},149,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20...,ἐπὶ Μενεκράτου ἄρχοντος ἐπὶ τῆς Οἰνεῖδος ἕκτη-...,Agora-XV.csv,231001,220/19
7001,/text/298501?location=1237&patt=&bookid=735&of...,"IDR III,1",153,Regions\nThrace and the Lower Danube (IG X)\nD...,"IDR III,1\n153",Dacia Sup. — Tibiscum (Jupa) — 2nd/3rd c. AD,{},8,1\n\n\n\n5\n\n\n,D(is) M(anibus)\nP(ublius) Ael(ius) Claudia-\n...,IDR-III-1.csv,298501,2nd/3rd c. AD
15498,/text/243501?location=1498&patt=&bookid=474&of...,IGLSyr 4,"1271B,g",Regions\nGreater Syria and the East\nSyria and...,"IGLSyr 4\n1271B,g","Syr., Laodik. — Laodicea — 12/1 BC",{},2,1\n,"ζλʹ {²sc. ἔτους}², ἑκκα-\nιδέκατον.",IGLSyr-4.csv,243501,12/1 BC
16567,/text/299501?location=1237&patt=&bookid=737&of...,"IDR III,3",182,Regions\nThrace and the Lower Danube (IG X)\nD...,"IDR III,3\n182",Dacia Sup. — Micia (Vețel)? — Chimindia — 2nd/...,{},9,frg. a\n\n\n\n\nfrg. b\n\n\n,[— — — — — — — — — — — —]\n[— —] quondam Pompo...,IDR-III-3.csv,299501,2nd/3rd c. AD
24698,/text/64501?location=915&patt=&bookid=1&offset...,ID,2057,"Regions\nAegean Islands, incl. Crete (IG XI-[X...",ID\n2057,Delos — 116/5 BC,{},6,1\n\n\n\n5\n,"Διονύσιος\nΔιονυσίου\nΣφή<τ>τιος, ἱερεὺς\nγενό...",ID.csv,64501,116/5 BC


# Parse centuries

In [39]:
centuries_df = get_as_dataframe(PHI_overview.worksheet("centuries"))
centuries_df.set_index("arabic", inplace=True)
centuries_df

,roman,start_BC,stop_BC,start_AD,stop_AD
arabic,,,,,
8th,VIII,-800,-701,701,800
7th,VII,-700,-601,601,700
6th,VI,-600,-501,501,600
4th,IV,-400,-301,301,400
5th,V,-500,401,401,500
3rd,III,-300,-201,201,300
2nd,II,-200,-101,101,200
1st,I,-100,-1,1,100


In [70]:
arabics = centuries_df.index.tolist()
arabics

['8th', '7th', '6th', '4th', '5th', '3rd', '2nd', '1st']

In [41]:
centuries_df["roman"].tolist()

['VIII', 'VII', 'VI', 'IV', 'V', 'III', 'II', 'I']

In [31]:
datation = "1st-early 2nd c. AD"
re.split("-|/|\sor\s", datation)

['1st', 'early 2nd c. AD']

In [43]:
centuries_df.loc["3rd"]["roman"]

'III'

In [48]:
datation = "s. II/III AD" # "p." and "a." are replaced previously
for roman, arabic in zip(centuries_df["roman"].tolist(), centuries_df.index.tolist()):
  datation = re.sub(roman, arabic, datation)
centuries = re.split("-|/|\sor\s", datation)
print(centuries)

['s. 2nd', '3rd AD']


In [84]:
any(re.search(arabic, "2nd c. AD") for arabic in arabics)

True

In [0]:
for num in arabics:
  century1 = [num for cents_list[0] if num in cents_list[0]]

In [103]:
num = "4th"
re.sub(".*" + num + ".", num, "beg. 4th c. ") # for num in arabics 

['s. 2nd c. ',
 's. 2nd c. ',
 's. 2nd c. ',
 's. 2nd c. ',
 's. 2nd c. ',
 's. 2nd c. ',
 's. 2nd',
 's. 2nd c. ']

In [0]:
### new version - still not functional...
def parse_centuries(datation, dating):
  for roman, arabic in zip(centuries_df["roman"].tolist(), arabics):
    datation = re.sub(roman, arabic, datation)
  cents = [cent for cent in centuries_df.index.tolist() if re.search(cent, datation)]
  # if more centuries mentioned:
  if len([cent for cent in centuries_df.index.tolist() if re.search(cent, datation)]) == 2:
    cents_list = re.split("-|/|\sor\s", datation)
    century1 = [re.sub(".*" + num + ".*", num, cents_list[0])  for num in arabics if num in string][0]
    century2 = [re.sub(".*" + num + ".*", num, cents_list[1])  for num in arabics if num in string][0]
    if (" AD" in datation) and (" BC" not in datation): # if explicit AD and only AD:
      start = centuries_df.loc[century1]["start_AD"]
      stop = centuries_df.loc[century2]["stop_AD"]
      era = "AD"
    elif (" BC" in cents_list[0]) and (" AD" in cents_list[1]):
      start = centuries_df.loc[century1]["start_BC"]
      stop = centuries_df.loc[century2]["stop_AD"]
      era = "BC/AD"
    else:
      start = centuries_df.loc[century1]["start_BC"]
      stop = centuries_df.loc[century2]["stop_BC"]
      era = "BC"
    dating_update = {"start" : start, "stop" : stop, "era" : era, "type" : "range+cent+morece"}
  elif len(cents) == 1:
    century = [re.sub(".*" + num + ".*", num, datation)  for num in arabics if num in string][0]
    if " AD" in datation: 
      start = centuries_df.loc[century]["start_AD"]
      stop = centuries_df.loc[century]["stop_AD"]
      era = "AD"
    else:
      start = centuries_df.loc[century]["start_BC"]
      stop = centuries_df.loc[century]["stop_BC"]
      era = "BC"
    dating_update = {"start" : start, "stop" : stop, "era" : era, "type" : "range+cent"}
  else:
    dating_update = {"type": "unknown"}
  return dating_update

In [118]:
arabics

['8th', '7th', '6th', '4th', '5th', '3rd', '2nd', '1st']

In [122]:
string = "s. 2nd"
century1 = [re.sub(".*" + num + ".*", num, string)  for num in arabics if num in string]
century1

['2nd']

In [120]:
datation = "s. II/III AD" # "p." and "a." are replaced previously
parse_centuries(datation, {"type": "unknown"})

KeyError: ignored

In [69]:
cents = [cent for cent in centuries_df.index.tolist() if re.search(cent, datation)]
cents_list = re.split("-|/|\sor\s", datation)
cents = [cent_num for cent_str in cents_list if cent_num in cent_str] # to correct order
cents


NameError: ignored

In [62]:
datation = "s. II/III AD"
for roman, arabic in zip(centuries_df["roman"].tolist(), centuries_df.index.tolist()):
  datation = re.sub(roman, arabic, datation)
cents = [cent for cent in centuries_df.index.tolist() if re.search(cent, datation)]
[cent for cent in cents

#parse_centuries(datation, {"type" : "unkwnown"})

['3rd', '2nd']

In [63]:
cents_list = re.split("-|/|\sor\s", datation)
cents_list

['s. 2nd', '3rd AD']

In [64]:
if (" AD" in datation) and (" BC" not in datation):
  print("yes")

yes


In [66]:
dating_update = {"start" : centuries_df.loc[cents[0]]["start_AD"], "stop" : centuries_df.loc[cents[-1]]["stop_AD"], "type" : "range+cent", "era" : "AD"}
dating_update

{'era': 'AD', 'start': 201, 'stop': 200, 'type': 'range+cent'}

In [0]:
cent_BC =                   [("s\.\sVI($|\s|\/)", -600, -501),
                             ("s\.\sV($|\s|\/)", -500, -401), 
                            ("s\.\sV/IV($|\s|\/)", -500, -301),
                            ("s\.\sIV($|\s|\/)", -400, -301),
                            ("s\.\sIII($|\s|\/)", -300, -201),
                            ("s\.\sII($|\s|\/)", -200, -101),
                            ("s\.\sI($|\s|\/)", -100, -1),
                            ("8th", -800, -701), 
                            ("7th", -700, -601),
                            ("6th", -600,  -501),
                            ("5th", -500,  -401),
                            ("4th", -400, -301),
                            ("3rd", -300, -201),
                            ("2nd", -200, -101),
                            ("1st", -100, -1)]
cent_AD =                  [("s\.\sI($|\s|\/)", 1, 100),
                            ("s\.\sII($|\s|\/)", 101, 200),
                            ("s\.\sIII($|\s|\/)", 201, 300),
                            ("s\.\sIV($|\s|\/)", 301, 400),
                            ("s\.\sV($|\s|\/)", 401, 500),
                            ("s\.\sV($|\s|\/)", 401, 500),
                            ("1st", 1, 100),
                            ("2nd", 101, 200),
                            ("3rd", 201, 300),
                            ("4th", 301, 400),
                            ("5th", 401, 500),
                            ("6th", 501, 600),
                            ("7th", 601, 700),
                            ("8th", 701, 800)]
def parse_centuries(datation, dating):
  #if (" BC" in datation) and (" AD" in datation):
  #  century_BC = [element_cent for element_cent in cent_BC if re.search(element_cent[0], datation.partition("BC")[0])]
  #  century_AD = [element_cent for element_cent in cent_AD if re.search(element_cent[0], datation.partition("AD")[2])]
  #  return {"start" : century_BC[0][1], "stop" : century_AD[-1][2], "type" : "range+cent", "era" : "BC/AD"} # use the first century for start date and last for stop date
  # else:
    try:
      if dating["era"] == "AD":
        dating_update = {}
        centuries_mentioned = [element_cent for element_cent in cent_AD if re.search(element_cent[0], datation)]
        return {"start" : centuries_mentioned[0][1], "stop" : centuries_mentioned[-1][2], "type" : "range+cent"} # use the first century for start date and last for stop date
      if dating["era"] == "BC":
        centuries_mentioned = [element_cent for element_cent in cent_BC if re.search(element_cent[0], datation)]
        return {"start" : centuries_mentioned[0][1], "stop" : centuries_mentioned[-1][2], "type" : "range+cent"} # use the first century for start date and last for stop date
    except:
      return {"type": "unknown"}

In [0]:
datation = "s. III/II"
for number_pair in [("VIII", "8th"),
                       ("VII", "7th"),
                       ("VI", "6th"),
                       ("IV", "4th"),
                       ("V", "5th"),
                       ("III", "3rd"),
                       ("II", "2nd"),
                       ("I", "1st")]:
  match = re.search("(s\.\s|/)(" + number_pair[0] + ")($|\s|/)", datation)
  if match:
    datation = re.sub("(s\.\s|/)(" + number_pair[0] + ")", number_pair[1] + match.groups()[2], datation)
datation

'3rd/2nd'

In [0]:
# example 1
parse_centuries("2nd/3rd c. AD", {"era": "AD"})

{'start': 101, 'stop': 300, 'type': 'range+cent'}

In [0]:
parse_centuries("2nd c. BC - 3rd c. AD", {"era": "AD"})

{'start': 101, 'stop': 300, 'type': 'range+cent'}

In [0]:
# ("a."  and "p." are replaced by "BC" and "AD" earlier in the script
parse_centuries("s. I BC/s. I AD", {"era": "BC/AD"})

In [0]:
parse_centuries("s. II/III a.", {"era": "BC"})

{'start': -200, 'stop': -101, 'type': 'range+cent'}

# Parse phase

In [0]:
def modify_by_phase(datation, dating):
  try:
    start, stop = dating["start"], dating["stop"]
    try: 
      duration = abs(dating["stop"] - dating["start"])
    except:
      duration = 1
    datation = datation.lower()
    if "early" in datation:
      coef = 0.25
      dating["stop"] = start + round(duration * coef)
      dating["type"] = dating["type"] + "+early"
    if "late" in datation:
      if "late antiquity" not in datation:
        coef = 0.25
        dating["start"] = stop - round(duration * coef)
        dating["type"] = dating["type"] + "+late"
    if re.search("(beginning|beg\.?\s)", datation):
      coef = 0.1
      dating["stop"] = start + round(duration * coef)
      dating["type"] = dating["type"] + "+beg"
    if re.search("(end\s|fin.\s)", datation):
      coef = 0.1
      dating["start"] = stop - round(duration * coef)
      dating["type"] = dating["type"] + "+end"
    if re.search("(middle|mid\.?\s|med\.\s)", datation):
      coef = 0.05 # that means: "middle 2nd c. AD" => 140 - 161
      dating_avr = (dating["start"] + dating["stop"]) / 2
      dating["start"] = round(dating_avr - (coef * duration))
      dating["stop"] = round(dating_avr + (coef * duration))
      dating["type"] = dating["type"] + "+middle"
    if re.search("ca\.\s", datation):
      dating["type"] = dating["type"] + "+ca"
      if ("exact" in dating["type"]) or duration < 10:
        dating.update({"start" : dating["exact"] - 5, "stop" : dating["exact"] + 5})
        dating["exact"] = None
      else:
        dating["start"] = start - round(duration * 0.1)
        dating["stop"] = stop + round(duration * 0.1)
    return dating
  except:
    return dating

In [0]:
datation = "ca. 200 BC"
dating = {"start" : None, "stop" : None, "exact" : -200, "type" : "exact", "era" : "BC"}
print(modify_by_phase(datation, dating))

{'start': -205, 'stop': -195, 'exact': None, 'type': 'exact+ca', 'era': 'BC'}


In [0]:
datation = "ca. s. II BC"
dating = {"start" : -200, "stop" : -101, "type" : "range+cent"}
print(modify_by_phase(datation, dating))

{'start': -210, 'stop': -91, 'type': 'range+cent+ca'}


In [0]:
datation = "[med. V a.]"
if datation[0] == "[":
  datation = datation[1:-1]
datation

'med. V a.'

In [0]:
datation = re.sub("a\.(\-|\/|$)", "AD", datation)

In [0]:
datation

'med. V AD'

In [0]:
datation =  "non ante med. s. II a."
datation = re.sub("(not\s(before|bef\.)\s|non\sante\s)", "post ", datation)
datation

'post med. s. II a.'

# Main function

In [0]:
def date_extractor(datation):
  dating = {"start": None, "stop": None, "exact": None, "or": None, "type": "unknown", "era": None}
  # UNCERTAINTY
  if "?" in datation:
    datation = datation.replace("?", "")
    dating["certainty"] = "?"
  if datation[0] == "[":
    datation = datation[1:-1]
    dating["certainty"] = "?"
  # PURE NUMBERS
  if dating["type"] == "unknown":
    match = re.match("^\s?(\d+)\s?$", datation) # if it is not just a number (numbers alone are good) 
    if match:
      dating.update({"exact" : int(match[0]), "type": "exact"})
  # ranges combining BC and AD:
  match = re.search("(\d+)(\s(a\.|BP))(\-)(\d+)(\s(p\.|AD))", datation) # (/sa/.)(/-)(/d+)(/sp/.)"
  if match:
    dating.update({"start" : int(match.groups()[0]) * -1, "stop" : match.groups()[4], "era" : "BC/AD", "type" : "range"})
  # "year/year"
  dating.update(match_or(datation, dating)) # find all "/" instances linked with individual years
  # PERIODS
  dating.update(extract_period(datation, dating))
  #replace " a." and " p." when at the end of datation or before "/" or "-"
  datation = re.sub("a\.(\-|\/|$)", "BC", datation)
  datation = re.sub("p\.(\-|\/|$)", "AD", datation)
  if dating["type"] == "unknown":
    if " AD" in datation:
            dating["era"] = "AD" 
            if dating["type"] == "unknown":
              try: dating.update(parse_centuries(datation, dating))
              except: pass
            ### try post quem and ante quem
            if dating["type"] == "unknown":
              dating.update(extract_ante_and_post(datation, dating))
            if dating["type"] == "unknown": 
              try:
                date_both = re.search('(\d+)(\-)(\d+)', datation, flags=re.IGNORECASE).groups()
                if any(time_indicator in datation for time_indicator in [" BC", " AD", "prob", "possi"]):
                  dating.update({"start" : int(date_both[0]), "stop" : int(date_both[2]), "type" : "range"})
              except:  
                  try:
                    match = re.search('\d+', datation, flags=re.IGNORECASE)
                    if any(time_indicator in datation for time_indicator in [" BC", " AD", "prob", "possi"]):
                      dating.update({"exact" : int(match[0]), "type": "exact"})
                  except:
                    pass
    ### try BC:
    else:
        dating["era"] = "BC"
        if dating["type"] == "unknown":
          try:
            dating.update(parse_centuries(datation, dating))
          except:
            pass
        ### try post quem and ante quem
        if dating["type"] == "unknown":
          dating.update(extract_ante_and_post(datation, dating))
        if dating["type"] == "unknown":
          try:
            date_both = re.search('(\d+)(\-)(\d+)', datation, flags=re.IGNORECASE).groups()
            if any(time_indicator in datation for time_indicator in [" BC", " AD", "prob", "possi"]):
              dating.update({"start" : int(date_both[0]) * -1, "stop" : int(date_both[2]) * -1, "type" : "range"})
          except:  
            try:
              date_exact = re.search('\d+', datation, flags=re.IGNORECASE)
              if any(time_indicator in datation for time_indicator in [" BC", " AD", "prob", "possi"]):
                dating.update({"exact" : int(date_exact.group(0)) * -1, "type": "exact"})
            except:
              pass
  # extract phases (e.g. "early", "middle", "late", "beginning" etc.)
  dating = modify_by_phase(datation, dating)
  # extract ante quem and post quem for ranges
  if "range" in dating["type"]:
    dating = extract_ante_and_post(datation, dating)
  if dating["type"]=="unknown":
    dating.update({"start": None, "stop": None, "exact": None, "or": None, "type": "unknown", "era" : None})
  return dating 

In [0]:
date_extractor("s. II/III AD")

{'era': 'AD',
 'exact': None,
 'or': None,
 'start': 101,
 'stop': 200,
 'type': 'range+cent'}

In [0]:
# testing/example
for datation in ["Byzantine", "non ante s. II a.", "140/39 BC", "Rom. Imp", "reign of Augustus", "ante 140 BC", "late Antonine period", "IosPE IV 348"]:
  print(datation, date_extractor(datation))

Byzantine {'start': None, 'stop': None, 'exact': None, 'or': None, 'type': 'unknown', 'era': None}
non ante s. II a. {'start': -201, 'stop': None, 'type': 'range+cent+post'}
140/39 BC {'start': -140, 'stop': -139, 'exact': None, 'or': None, 'type': 'range', 'era': 'BC'}
Rom. Imp {'start': -31, 'stop': 410, 'exact': None, 'or': None, 'type': 'range+period', 'era': 'BC/AD', 'source': 'PeriodO', 'notes': nan, 'link': 'http://n2t.net/ark:/99152/p08m57hqcc5'}
reign of Augustus {'start': -27, 'stop': 14, 'exact': None, 'or': None, 'type': 'range+period', 'era': 'BC/AD', 'source': 'PeriodO', 'notes': nan, 'link': 'http://n2t.net/ark:/99152/p06c6g3xnmx'}
ante 140 BC {'start': None, 'stop': -140, 'exact': None, 'or': None, 'type': 'ante', 'era': 'BC'}
late Antonine period {'start': 168, 'stop': 192, 'exact': None, 'or': None, 'type': 'range+period+late', 'era': 'AD', 'source': 'PeriodO', 'notes': nan, 'link': 'http://n2t.net/ark:/99152/p06c6g34zjk'}
IosPE IV 348 {'start': None, 'stop': None, 'e

In [0]:
datation = "3rd c. AD"
date_extractor(datation)

{'era': 'AD',
 'exact': None,
 'or': None,
 'start': 201,
 'stop': 300,
 'type': 'range+cent'}

In [0]:
datation = "Nécrop.Myr. 224,5"
date_extractor(datation)

{'era': None,
 'exact': None,
 'or': None,
 'start': None,
 'stop': None,
 'type': 'unknown'}

In [0]:
 datation = " non ante med. s. II p."
 date_extractor(datation)

{'start': 145, 'stop': None, 'type': 'range+cent+middle+post'}

In [0]:
date_extractor("not bef. the Antonine period")

{'start': 95, 'stop': None, 'type': 'range+period+post'}

In [0]:
datation = "27 a.-14 p."
match = re.search("(\d+)(\s(a\.|BP))(\-)(\d+)(\s(p\.|AD))", datation) # (/sa/.)(/-)(/d+)(/sp/.)"
if match:
  dating = {"start" : int(match.groups()[0]) * -1, "stop" : match.groups()[4], "era" : "BC/AD", "type" : "range"}
dating

{'era': 'BC/AD', 'start': -27, 'stop': '14', 'type': 'range'}

In [0]:
test_list = PHI_by_500["raw_date"].tolist()
for datation in test_list:
  print(datation, date_extractor(datation))

IStr 427 {'start': None, 'stop': None, 'exact': None, 'or': None, 'type': 'unknown', 'era': None}
 Rom. Imp. period {'start': -31, 'stop': 410, 'exact': None, 'or': None, 'type': 'range+period', 'era': 'BC/AD', 'source': 'PeriodO', 'notes': nan, 'link': 'http://n2t.net/ark:/99152/p08m57hqcc5'}
 220/19 {'start': -220, 'stop': -219, 'exact': None, 'or': None, 'type': 'range', 'era': 'BC'}
Thasos {'start': None, 'stop': None, 'exact': None, 'or': None, 'type': 'unknown', 'era': None}
 el-Boueib {'start': None, 'stop': None, 'exact': None, 'or': None, 'type': 'unknown', 'era': None}
 ca. 100-150 AD {'start': 95, 'stop': 155, 'exact': None, 'or': None, 'type': 'range+ca', 'era': 'AD'}
 267-668 AD {'start': 267, 'stop': 668, 'exact': None, 'or': None, 'type': 'range', 'era': 'AD'}
 1st c. BC  {'start': -100, 'stop': -1, 'exact': None, 'or': None, 'type': 'range+cent', 'era': 'BC'}
 Roman period?  {'start': -146, 'stop': 324, 'exact': None, 'or': None, 'type': 'range+period', 'era': 'BC/AD', 

In [0]:
PHI_list_of_dict = []
for inscription_date_tuple in list(zip(PHI_by_500["tildeinfo"].tolist(), PHI_by_500["PHI_ID"].tolist(),  PHI_by_500["raw_date"].tolist())):
  dating = date_extractor(inscription_date_tuple[2])
  data_dict = {"tildeinfo": inscription_date_tuple[0], "PHI_ID" : inscription_date_tuple[1], "raw_date" : inscription_date_tuple[2]}
  data_dict.update(dating)
  PHI_list_of_dict.append(data_dict)

In [0]:
PHI_by_500_dates_v6 = pd.DataFrame(PHI_list_of_dict)
PHI_by_500_dates_v6.head(5)

,tildeinfo,PHI_ID,raw_date,start,stop,exact,or,type,era,source,notes,link,certainty
0,IStr 427,262001,IStr 427,NaN,None,NaN,NaN,unknown,None,NaN,NaN,NaN,NaN
1,Pont. — Amasia — Rom. Imp. period,266001,Rom. Imp. period,-31.0,410,NaN,NaN,range+period,BC/AD,PeriodO,NaN,http://n2t.net/ark:/99152/p08m57hqcc5,NaN
2,Att. — Athens: Agora — 220/19,231001,220/19,-220.0,-219,NaN,NaN,range,BC,NaN,NaN,NaN,NaN
3,Thasos,79501,Thasos,NaN,None,NaN,NaN,unknown,None,NaN,NaN,NaN,NaN
4,Eg. — el-Boueib,218501,el-Boueib,NaN,None,NaN,NaN,unknown,None,NaN,NaN,NaN,NaN


In [0]:
set_with_dataframe(PHI_overview.add_worksheet("PHI_by_500_dates_v6", 1,1), PHI_by_500_dates_v6)

# Match "/" for individual dates

In [0]:
def match_or(datation):
  match = re.search(r'(\d+)(\/)(\d+)', str(datation), flags=re.IGNORECASE)
  if match != None:
      date1 = match.groups()[0]
      date2 = match.groups()[2]
      len_diff = len(date1) - len(date2)
      if len_diff > 0:
        date2 = date1[:len_diff] + date2
      date1 = int(date1)
      date2 = int(date2)
      if "AD" not in datation:
        date1 = int(date1) * -1
        date2 = {int(date2) * -1
      return {"exact" : date1, "or": date2, "type" : "or"}
  else:
      return {"type" : "unknown"}


In [0]:
ors = PHI_by_500[PHI_by_500["raw_date"].str.contains(r'(\d+)(\/)(\d+)')]["raw_date"].tolist()
print(ors)

[' 220/19', ' 12/1 BC', ' 116/5 BC', ' 229/30 or 230/1 AD ', ' ante 336/5', ' 204/3?', ' 73/4 AD', ' ca. 163/2 BC', ' 128/127 BC ', ' shortly after 208/7 BC (or 207/6) ', ' 154/5 AD', ' 66/7 AD']


/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [0]:
# example/testing
for our_or in ors:
  print({our_or: match_or(our_or)})


{' 220/19': {'exact': -220, 'or': -219, 'type': 'or'}}
{' 12/1 BC': {'exact': -12, 'or': -11, 'type': 'or'}}
{' 116/5 BC': {'exact': -116, 'or': -115, 'type': 'or'}}
{' 229/30 or 230/1 AD ': {'exact': 229, 'or': 230, 'type': 'or'}}
{' ante 336/5': {'exact': -336, 'or': -335, 'type': 'or'}}
{' 204/3?': {'exact': -204, 'or': -203, 'type': 'or'}}
{' 73/4 AD': {'exact': 73, 'or': 74, 'type': 'or'}}
{' ca. 163/2 BC': {'exact': -163, 'or': -162, 'type': 'or'}}
{' 128/127 BC ': {'exact': -128, 'or': -127, 'type': 'or'}}
{' shortly after 208/7 BC (or 207/6) ': {'exact': -208, 'or': -207, 'type': 'or'}}
{' 154/5 AD': {'exact': 154, 'or': 155, 'type': 'or'}}
{' 66/7 AD': {'exact': 66, 'or': 67, 'type': 'or'}}


In [0]:
ors = PHI_by_500[PHI_by_500["raw_date"].str.contains("(\d+)(\/)(\d+)")]["raw_date"].tolist()
print(ors)

[' 220/19', ' 12/1 BC', ' 116/5 BC', ' 229/30 or 230/1 AD ', ' ante 336/5', ' 204/3?', ' 73/4 AD', ' ca. 163/2 BC', ' 128/127 BC ', ' shortly after 208/7 BC (or 207/6) ', ' 154/5 AD', ' 66/7 AD']


/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


# Under development

In [0]:
PHI_by_500[PHI_by_500["raw_date"].str.contains("/")]

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID,raw_date
1260,/text/231001?location=1365&patt=&bookid=394&of...,Agora XV,130,Regions\nAttica (IG I-III)\nAttica,Agora XV\n130,Att. — Athens: Agora — 220/19,{},149,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20...,ἐπὶ Μενεκράτου ἄρχοντος ἐπὶ τῆς Οἰνεῖδος ἕκτη-...,Agora-XV.csv,231001,220/19
7001,/text/298501?location=1237&patt=&bookid=735&of...,"IDR III,1",153,Regions\nThrace and the Lower Danube (IG X)\nD...,"IDR III,1\n153",Dacia Sup. — Tibiscum (Jupa) — 2nd/3rd c. AD,{},8,1\n\n\n\n5\n\n\n,D(is) M(anibus)\nP(ublius) Ael(ius) Claudia-\n...,IDR-III-1.csv,298501,2nd/3rd c. AD
15498,/text/243501?location=1498&patt=&bookid=474&of...,IGLSyr 4,"1271B,g",Regions\nGreater Syria and the East\nSyria and...,"IGLSyr 4\n1271B,g","Syr., Laodik. — Laodicea — 12/1 BC",{},2,1\n,"ζλʹ {²sc. ἔτους}², ἑκκα-\nιδέκατον.",IGLSyr-4.csv,243501,12/1 BC
16567,/text/299501?location=1237&patt=&bookid=737&of...,"IDR III,3",182,Regions\nThrace and the Lower Danube (IG X)\nD...,"IDR III,3\n182",Dacia Sup. — Micia (Vețel)? — Chimindia — 2nd/...,{},9,frg. a\n\n\n\n\nfrg. b\n\n\n,[— — — — — — — — — — — —]\n[— —] quondam Pompo...,IDR-III-3.csv,299501,2nd/3rd c. AD
24698,/text/64501?location=915&patt=&bookid=1&offset...,ID,2057,"Regions\nAegean Islands, incl. Crete (IG XI-[X...",ID\n2057,Delos — 116/5 BC,{},6,1\n\n\n\n5\n,"Διονύσιος\nΔιονυσίου\nΣφή<τ>τιος, ἱερεὺς\nγενό...",ID.csv,64501,116/5 BC
35543,/text/217001?location=1497&patt=&bookid=362&of...,"Bernand, Inscr. Métr.",113,"Regions\nEgypt, Nubia and Cyrenaïca\nEgypt and...","Bernand, Inscr. Métr.\n113",Eg. — Naukratis (Kōm Giéif) — 3rd/2nd c. BC?,{},4,1\n\n\n,Νειλούσσης ἀλόχου τήνδ’ εἰκόνα Παρθενοπαί[ου]\...,Bernand--Inscr.-M-tr..csv,217001,3rd/2nd c. BC?
38001,/text/228501?location=1497&patt=&bookid=389&of...,Delta I,"662,198","Regions\nEgypt, Nubia and Cyrenaïca\nEgypt and...","Delta I\n662,198",Eg. — Naukratis — 6th/5th c. BC,{},1,1,Κ̣α̣ρό̣φνης με ἀνέθηκε τἀπό̣[λλοˉνι το͂ι Μ]ιλα...,Delta-I.csv,228501,6th/5th c. BC
38501,/text/229001?location=1497&patt=&bookid=389&of...,Delta I,"712,699","Regions\nEgypt, Nubia and Cyrenaïca\nEgypt and...","Delta I\n712,699",Eg. — Naukratis — 6th/5th c. BC,{},1,1,[Ἀ]πόλλ[ωνός ἐˉμι].,Delta-I.csv,229001,6th/5th c. BC
58576,/text/291501?location=1&patt=&bookid=172&offse...,SEG,21:506,Regions,SEG\n21:506,Att. — 229/30 or 230/1 AD — IG II² 1064+,{},69,\n\n\nfrg. a.1\n\n\n\n5\n\n\n\n\n10\nfrg. b.10...,"IG II(2) 1064+, Oliver, Hesp. Suppl. VI (1941)...",SEG.csv,291501,229/30 or 230/1 AD
60698,/text/152001?location=1&patt=&bookid=172&offse...,SEG,26:759,Regions,SEG\n26:759,Makedonia (Mygdonia) — Thessalonike — 2nd/3rd ...,{},3,6\n\n,. . . εἰ δέ\n[τις ἕτ]ε̣ρος κα-\n[ταθῇ ἕτερον π...,SEG.csv,152001,2nd/3rd c. AD


In [0]:
re.match("")

In [0]:
set_with_dataframe(PHI_overview.add_worksheet("PHI_by_500_dates_v1", 1, 1), PHI_by_500)